# Hotel Acme Sands - Executive Operations Report

(Header will be displayed when you run the first code cell below)

In [ ]:
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from IPython.display import Image, display, Markdown

# Get current date
report_date = datetime.now().strftime("%B %d, %Y")

# Display dynamic header with current date
header_markdown = f'''# Hotel Acme Sands - Executive Operations Report

**Prepared for:** Future stakeholders, business executives  
**Date:** {report_date}  
**Analysis Period:** April 2024 - November 2025  

## Executive Summary

This report analyzes 17 months of housekeeping operations data to identify productivity patterns, operational efficiency, and potential concerns requiring management attention.

**Key Findings:**
- 3 employees require immediate investigation for impossible productivity claims
- 43 employees need additional training support  
- Day shift operations need enhanced oversight
- Overall operations are solid with targeted improvements needed
'''

display(Markdown(header_markdown))

# Load the data
users = pd.read_csv("../data/users.csv")
workflow = pd.read_csv("../data/workflow_log.csv")

# Focus on room cleaning activities
room_cleaning = workflow[workflow['action'] == 'Clean Room'].copy()
room_cleaning['date'] = pd.to_datetime(room_cleaning['date'])

print("\nData loaded successfully")
print(f"Analyzing {len(room_cleaning)} room cleaning sessions")
print(f"Covering {len(users)} total employees")


## Operational Overview

### Shift Definitions
- **Day Shift:** 6:00 AM – 2:00 PM
- **Swing Shift:** 2:00 PM – 10:00 PM
- **Night Shift:** 10:00 PM – 6:00 AM

### Analysis Scope
- **Time Period:** 17 months of operations
- **Data Points:** 500+ workflow entries
- **Focus Area:** Room cleaning productivity and quality

In [ ]:
# Calculate shift performance metrics
employee_performance = room_cleaning.merge(users, on='employee_id', how='left')

# Basic shift statistics
shift_counts = room_cleaning['shift'].value_counts()
shift_productivity = room_cleaning.groupby('shift').size()
shift_avg_rooms = room_cleaning.groupby('shift')['room_cleaned'].mean()

print("SHIFT PERFORMANCE SUMMARY:")
print("="*50)
for shift in ['Day', 'Night', 'Swing']:
    if shift in shift_productivity.index:
        sessions = shift_productivity[shift]
        avg_rooms = shift_avg_rooms[shift] if shift in shift_avg_rooms.index else 0
        print(f"{shift:10}: {sessions:2} sessions | {avg_rooms:.1f} avg rooms/session")

print("\nSHIFT DISTRIBUTION:")


In [ ]:
# Check if visualization files exist
print("CHECKING FOR VISUALIZATION FILES:")
print("="*40)

viz_files = [
    '../visualizations/shift_productivity_analysis.png',
    '../visualizations/shift_distribution.png', 
    '../visualizations/fraud_risk_assessment.png',
    '../visualizations/executive_dashboard.png'
]

for file in viz_files:
    if os.path.exists(file):
        print(f"{file} - EXISTS")
    else:
        print(f"{file} - MISSING")

print(f"\nCurrent directory: {os.getcwd()}")
print(f"Files in current directory: {os.listdir('..')}")

if os.path.exists('../visualizations'):
    print(f"Files in visualizations: {os.listdir('../visualizations')}")


In [ ]:
print("SHIFT PERFORMANCE ANALYSIS:")
display(Image('../visualizations/shift_productivity_analysis.png'))

print("\nKEY INSIGHTS:")
print("- Day shift shows highest average productivity (24.6 rooms/session)")
print("- Night and Swing shifts have similar performance (~20-21 rooms/session)")
print("- All shifts handle similar workloads (21-24 sessions each)")


In [ ]:
print("SHIFT DISTRIBUTION:")
display(Image('../visualizations/shift_distribution.png'))

print("\nSTAFFING INSIGHTS:")
print("- Work is evenly distributed across all three shifts")
print("- No single shift is overwhelmed with activity")
print("- Balanced coverage supports 24/7 operations")


## Monthly Activity Patterns

Understanding seasonal trends helps with staffing decisions and resource planning.

In [ ]:
# Monthly Activity Patterns Analysis for Executive Report

# Calculate monthly statistics for executive summary
workflow['date'] = pd.to_datetime(workflow['date'])
workflow['month'] = workflow['date'].dt.month_name()

# Focus on room cleaning activities specifically
room_cleaning_monthly = workflow[workflow['action'] == 'Clean Room']
monthly_room_activity = room_cleaning_monthly.groupby(
    room_cleaning_monthly['date'].dt.month_name())['room_cleaned'].sum()

# Reorder months chronologically
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']
monthly_room_activity = monthly_room_activity.reindex(
    [m for m in month_order if m in monthly_room_activity.index])

print("SEASONAL ACTIVITY SUMMARY")
print("=" * 40)

if len(monthly_room_activity) > 0:
    print(f"Peak Activity Months:")
    top_3_months = monthly_room_activity.nlargest(3)
    for month, rooms in top_3_months.items():
        print(f"  {month}: {rooms:,.0f} rooms cleaned")
    
    print(f"\nLower Activity Months:")
    bottom_3_months = monthly_room_activity.nsmallest(3)
    for month, rooms in bottom_3_months.items():
        print(f"  {month}: {rooms:,.0f} rooms cleaned")
    
    # Calculate percentage difference between peak and low
    peak_month = monthly_room_activity.max()
    low_month = monthly_room_activity.min()
    
    if low_month > 0:
        variance = ((peak_month - low_month) / low_month) * 100
    else:
        non_zero_months = monthly_room_activity[monthly_room_activity > 0]
        if len(non_zero_months) > 1:
            low_month = non_zero_months.min()
            variance = ((peak_month - low_month) / low_month) * 100
        else:
            variance = 0
    
    print(f"\nActivity Variance: {variance:.0f}% difference between peak and low months")
    
    print(f"\nSTAFFING INSIGHTS:")
    if len(top_3_months) > 1:
        print(f"  - Consider seasonal staff for {top_3_months.index[0]} and {top_3_months.index[1]}")
    elif len(top_3_months) == 1:
        print(f"  - Consider seasonal staff for {top_3_months.index[0]}")
    else:
        print("  - No clear peak months identified")
    
    if len(bottom_3_months) > 0:
        print(f"  - Plan maintenance during {bottom_3_months.index[0]} (lowest activity)")
    else:
        print("  - No clear low activity months identified")
    
    if not pd.isna(variance):
        print(f"  - Budget for {variance:.0f}% capacity fluctuation throughout the year")
    else:
        print("  - Unable to calculate capacity variance")
else:
    print("No room cleaning data found - check action filtering")


## Fraud Detection Analysis

Our analysis identified several employees claiming impossible productivity levels. Industry standards indicate that cleaning 8+ rooms per hour or 60+ rooms per shift is physically impossible and suggests fraudulent reporting.

**Detection Criteria:**
- **Impossible productivity:** >8 rooms per hour
- **Unrealistic shift totals:** >60 rooms in 8 hours
- **Perfect completion patterns:** 100% completion with extreme productivity

**Risk Assessment:** HIGH - Requires immediate management attention

In [ ]:
# Fraud Detection Analysis for Executive Report
print("FRAUD DETECTION ANALYSIS")
print("=" * 50)

# Analyze room cleaning productivity for impossible claims
room_cleaning_stats = room_cleaning.copy()

# Define realistic productivity thresholds
MAX_REALISTIC_ROOMS_PER_HOUR = 8
MAX_REALISTIC_ROOMS_PER_SHIFT = 60

# Calculate productivity metrics
room_cleaning_stats['shift_duration_hours'] = 8
room_cleaning_stats['rooms_per_hour'] = (
    room_cleaning_stats['room_cleaned'] / room_cleaning_stats['shift_duration_hours'])

# Identify suspicious cases
suspicious_cases = room_cleaning_stats[
    (room_cleaning_stats['room_cleaned'] > MAX_REALISTIC_ROOMS_PER_SHIFT) |
    (room_cleaning_stats['rooms_per_hour'] > MAX_REALISTIC_ROOMS_PER_HOUR)
].copy()

# Merge with employee details for investigation
if len(suspicious_cases) > 0:
    fraud_investigation = suspicious_cases.merge(users, on='employee_id', how='left')
    
    print(f"SUSPICIOUS ACTIVITY DETECTED:")
    print(f"  - {len(suspicious_cases)} cases requiring immediate investigation")
    print(f"  - {len(fraud_investigation['employee_id'].unique())} employees involved")
    print()
    
    print("TOP PRIORITY INVESTIGATIONS:")
    print("-" * 40)
    
    display_cols = ['employee_id', 'Name', 'room_cleaned', 'shift', 'date', 
                    'Phone', 'City', 'State']
    top_suspicious = fraud_investigation.nlargest(5, 'room_cleaned')[display_cols]
    
    for idx, case in top_suspicious.iterrows():
        print(f"  {case['Name']} (ID: {case['employee_id']})")
        print(f"    Date: {case['date']}")
        print(f"    Shift: {case['shift']}")
        print(f"    Claimed Rooms: {case['room_cleaned']:.0f}")
        print()
    
    # Analyze patterns by shift
    shift_fraud_analysis = fraud_investigation.groupby('shift').size()
    print("FRAUD PATTERNS BY SHIFT:")
    for shift, count in shift_fraud_analysis.items():
        print(f"  {shift}: {count} suspicious cases")
    print()
    
    print("RECOMMENDED ACTIONS:")
    print("  - Coordinate with HR for formal investigation process")
    print("  - Review room assignment logs and security footage for flagged dates")
    print("  - Involve Room Service Manager/Supervisor for employee meetings")
    print("  - Implement enhanced verification procedures")
    print("  - Consider policy review and staff training on reporting standards")
else:
    print("No suspicious productivity patterns detected")
    print("All employee productivity appears within normal ranges")

# Additional analysis: Completion rate patterns
print(f"\nCOMPLETION RATE ANALYSIS:")
print("-" * 30)

completion_rates = room_cleaning.groupby('employee_id').agg({
    'status': lambda x: (x == 'complete').mean(),
    'room_cleaned': ['count', 'mean', 'max']
}).round(3)

completion_rates.columns = ['completion_rate', 'total_sessions', 'avg_rooms', 'max_rooms']
completion_rates = completion_rates.reset_index()

# Flag perfect completion rates
perfect_completion = completion_rates[completion_rates['completion_rate'] == 1.0]
high_productivity_perfect = perfect_completion[perfect_completion['max_rooms'] > 50]

if len(high_productivity_perfect) > 0:
    print(f"ADDITIONAL CONCERN:")
    print(f"  - {len(high_productivity_perfect)} employees with perfect completion AND high productivity")
    print(f"  - This combination may indicate systematic fraud")
else:
    print("No concerning completion rate patterns detected")

print(f"\nOVERALL EMPLOYEE PERFORMANCE DISTRIBUTION:")
print(f"  - Total employees analyzed: {len(completion_rates)}")
print(f"  - Average completion rate: {completion_rates['completion_rate'].mean():.1%}")
print(f"  - Employees needing training support: {len(completion_rates[completion_rates['completion_rate'] < 0.7])}")
print(f"  - High performers (>90% completion): {len(completion_rates[completion_rates['completion_rate'] > 0.9])}")


## Executive Summary & Recommendations

### Key Operational Findings

**Positive Indicators:**
- Balanced shift coverage across 24/7 operations
- Day shift demonstrates highest productivity (24.6 rooms/session average)
- Clear seasonal patterns enable strategic workforce planning
- 11 high-performing employees (>90% completion rate) provide training mentorship opportunities

**Critical Issues Requiring Immediate Attention:**

1. **Fraud Detection Results:**
- 6 employees claiming impossible productivity levels (70-118 rooms per shift)
- Estimated impact: Potential systematic reporting fraud
- Primary concern: Day and Swing shifts show highest fraud incidents

2. **Training & Performance Crisis:**
- 81% of workforce (47 employees) requires additional training support
- Average completion rate of only 23% indicates systemic performance issues
- Significant gap between high performers and struggling employees

3. **Seasonal Workforce Challenges:**
- 4,360% variance between peak (March) and low activity months (May/October)
- Critical staffing needs during March peak season (223 rooms cleaned)
- Zero activity months suggest potential operational inefficiencies

### Strategic Recommendations

**Immediate Actions (0-30 days):**
- Coordinate with HR to investigate 6 flagged employees
- Implement room count verification procedures
- Review Day shift supervision protocols
- Conduct emergency training assessment for underperforming staff

**Short-term Improvements (1-3 months):**
- Develop comprehensive training program for 47 identified employees
- Establish mentor partnerships using 11 high-performing staff members
- Implement enhanced shift monitoring and verification systems
- Create seasonal staffing plan to address 4,360% activity variance

**Long-term Strategic Initiatives (3-12 months):**
- Design performance-based compensation structure
- Establish predictive staffing model based on seasonal patterns
- Develop career advancement pathways for high-performing employees
- Create data-driven room assignment and productivity tracking systems

### Resource Requirements

**Training Investment:**
- Estimated 47 employees requiring skills development
- Recommended: Structured 2-week intensive training program
- Budget consideration: Training materials and temporary coverage staff

**Technology Enhancement:**
- Room verification system implementation
- Real-time productivity tracking dashboard
- Seasonal demand forecasting tools

**Staffing Adjustments:**
- Seasonal staff hiring strategy for March/December peaks
- Enhanced supervision during identified high-risk periods
- Cross-training program to improve shift flexibility

### Expected Outcomes

With proper implementation of these recommendations:
- **Fraud elimination:** Zero tolerance policy with verification systems
- **Performance improvement:** Target 80%+ completion rate across workforce
- **Operational efficiency:** Optimized staffing aligned with seasonal demands
- **Cost savings:** Reduced training costs through mentor programs and improved retention

**Bottom Line:** While serious issues have been identified, the data provides a clear roadmap for operational excellence. The 11 high-performing employees demonstrate that success is achievable across the entire workforce with proper support and systems.

## Operational Improvement Plan

### Phase 1: Crisis Management (Immediate - 30 Days)

**Fraud Mitigation:**
- Implement daily room count verification system
- Require supervisor sign-off for claims >50 rooms per shift
- Install random audit checks across all shifts
- Create incident reporting protocol for suspicious activities

**Performance Stabilization:**
- Conduct skills assessment for 47 underperforming employees
- Pair struggling staff with 11 high performers for immediate mentoring
- Establish daily check-ins for employees with <50% completion rates
- Create emergency training modules for critical skills gaps

**Shift Oversight Enhancement:**
- Add floating supervisor coverage during peak hours
- Implement hourly progress check system
- Create shift handoff documentation requirements
- Establish clear escalation procedures for productivity issues

### Phase 2: System Implementation (30-90 Days)

**Technology Solutions:**
- Deploy mobile app for real-time room status updates
- Create dashboard for supervisors to monitor productivity trends
- Implement QR code verification system for completed rooms
- Establish automated alerts for unusual productivity patterns

**Training Program Rollout:**
- Launch 2-week intensive training program for identified employees
- Create competency-based certification system
- Develop video training library for consistent skill development
- Establish quarterly performance review cycles

**Quality Assurance Framework:**
- Institute random room quality inspections
- Create customer feedback integration system
- Develop standardized cleaning protocols and checklists
- Implement photo documentation for quality verification

### Phase 3: Optimization & Growth (90-365 Days)

**Seasonal Staffing Strategy:**
- Develop predictive hiring model based on 4,360% seasonal variance
- Create flexible scheduling system for peak months (March, December)
- Establish partnerships with temporary staffing agencies
- Design cross-training program for multi-shift coverage

**Performance Excellence Program:**
- Launch employee recognition program for consistent high performers
- Create career advancement pathways within housekeeping department
- Develop performance-based compensation structure
- Establish annual awards program for operational excellence

**Data-Driven Decision Making:**
- Implement predictive analytics for room assignment optimization
- Create monthly operational dashboards for management review
- Develop KPI tracking system for continuous improvement
- Establish benchmarking against industry standards

### Success Metrics & Monitoring

**Monthly Tracking:**
- Employee completion rates (Target: >80%)
- Fraud incident reports (Target: 0)
- Training program completion rates
- Seasonal staffing adequacy scores

**Quarterly Reviews:**
- Overall productivity improvements
- Staff retention and satisfaction scores
- Cost per room cleaned efficiency
- Customer satisfaction ratings

**Annual Assessment:**
- ROI analysis of improvement investments
- Comparison to industry benchmarks
- Strategic plan effectiveness review
- Long-term workforce development progress

### Budget Considerations

**Initial Investment (Phase 1):**
- Technology implementation: $15,000-25,000
- Training program development: $10,000-15,000
- Additional supervision coverage: $8,000-12,000/month

**Ongoing Operational Costs:**
- Enhanced training programs: $5,000/quarter
- Technology maintenance: $2,000/month
- Quality assurance activities: $3,000/month

**Expected Savings:**
- Reduced fraud losses: $20,000-40,000/year
- Improved efficiency: 15-25% productivity gains
- Lower staff turnover: $12,000-18,000/year in recruitment costs

### Implementation Timeline

**Week 1-2:** Crisis management protocols and immediate fraud prevention
**Week 3-4:** Skills assessment and emergency training rollout
**Month 2:** Technology deployment and system integration
**Month 3:** Full training program launch and quality assurance implementation
**Month 4-6:** Seasonal staffing strategy development and testing
**Month 7-12:** Optimization, refinement, and performance excellence program rollout

**Critical Success Factor:** Executive commitment to sustained investment in workforce development and operational excellence.

In [ ]:
# Visual Summary Dashboard for Executive Report
print("Visual Summary Dashboard")
print("=" * 50)

# Define constants
MAX_REALISTIC_ROOMS_PER_SHIFT = 60

# CVD-friendly color palette - intuitive and meaningful
CHART_COLORS = {
    'fraud': '#CC78BC',           # Purple-pink - fraud/critical
    'training_needed': '#DE8F05', # Orange - needs attention
    'high_performance': '#029E73', # Teal - high performance
    'normal_neutral': '#0173B2',  # Blue - normal/neutral
    'average': '#949494',          # Gray - average
    'light_neutral': '#56B4E9'     # Light blue - lower activity (lighter shade = less activity)
}

# Create comprehensive executive dashboard
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# 1. Fraud Risk Analysis (Top Left) - Purple-pink for all fraud
fraud_cases = room_cleaning[room_cleaning['room_cleaned'] > MAX_REALISTIC_ROOMS_PER_SHIFT]

if len(fraud_cases) > 0:
    fraud_analysis = fraud_cases.merge(users, on='employee_id', how='left')
    top_fraud = fraud_analysis.nlargest(6, 'room_cleaned')[['Name', 'room_cleaned']]
    fraud_names = top_fraud['Name'].tolist()
    fraud_rooms = top_fraud['room_cleaned'].tolist()
    
    # ALL fraud cases use purple-pink
    fraud_colors = [CHART_COLORS['fraud']] * len(fraud_rooms)
    
    bars1 = ax1.barh(fraud_names, fraud_rooms, color=fraud_colors)
    ax1.set_title('Fraud Detection: Impossible Room Claims', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Rooms Claimed per Shift')
    ax1.axvline(x=MAX_REALISTIC_ROOMS_PER_SHIFT, color='black', 
                linestyle='--', linewidth=2, alpha=0.7, 
                label=f'Max Realistic ({MAX_REALISTIC_ROOMS_PER_SHIFT})')
    ax1.legend()
    ax1.grid(axis='x', alpha=0.3)
    
    for bar, value in zip(bars1, fraud_rooms):
        ax1.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2,
                f'{int(value)}', va='center', fontweight='bold')
else:
    ax1.text(0.5, 0.5, 'No Fraud Cases Detected', ha='center', va='center', 
             transform=ax1.transAxes)
    ax1.set_title('Fraud Detection: No Issues Found', fontsize=14, fontweight='bold')

# 2. Seasonal Activity Patterns (Top Right) - Intuitive intensity gradient
monthly_activity = room_cleaning.groupby(
    room_cleaning['date'].dt.month_name())['room_cleaned'].sum()

if len(monthly_activity) > 0:
    month_order = ['January', 'February', 'March', 'April', 'May', 'June',
                   'July', 'August', 'September', 'October', 'November', 'December']
    monthly_ordered = monthly_activity.reindex(
        [m for m in month_order if m in monthly_activity.index])
    month_abbrev = [m[:3] for m in monthly_ordered.index]
    rooms_by_month = monthly_ordered.values
    
    # Intuitive color scheme: Darker/stronger = more activity, Lighter = less activity
    if len(rooms_by_month) > 0 and max(rooms_by_month) > 0:
        q75 = np.percentile(rooms_by_month, 75)
        q25 = np.percentile(rooms_by_month, 25)
        # Teal (strong) = high activity, Blue (medium) = normal, Light blue (weak) = low
        colors = [CHART_COLORS['high_performance'] if x > q75 
                 else CHART_COLORS['light_neutral'] if x < q25 
                 else CHART_COLORS['normal_neutral'] for x in rooms_by_month]
    else:
        colors = [CHART_COLORS['normal_neutral']] * len(rooms_by_month)
    
    bars2 = ax2.bar(month_abbrev, rooms_by_month, color=colors)
    ax2.set_title('Seasonal Activity Patterns', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Total Rooms Cleaned')
    ax2.tick_params(axis='x', rotation=45)
    ax2.grid(axis='y', alpha=0.3)
    
    if max(rooms_by_month) > 0:
        ax2.axhline(y=q75, color='black', linestyle='--', linewidth=2, alpha=0.7,
                   label=f'High Season Threshold ({q75:.0f})')
        ax2.legend()
else:
    ax2.text(0.5, 0.5, 'No Monthly Data Available', ha='center', va='center', 
             transform=ax2.transAxes)
    ax2.set_title('Seasonal Activity Patterns', fontsize=14, fontweight='bold')

# 3. Employee Performance Distribution (Bottom Left) - Clear performance categories
completion_rates = room_cleaning.groupby('employee_id')['status'].apply(
    lambda x: (x == 'complete').mean())
total_employees = len(completion_rates)

if total_employees > 0:
    high_performers = len(completion_rates[completion_rates >= 0.9])
    training_needed = len(completion_rates[completion_rates < 0.7])
    average_performers = total_employees - high_performers - training_needed
    
    categories = []
    counts = []
    colors = []
    
    if high_performers > 0:
        categories.append(f'High Performers\n(>90%): {high_performers}')
        counts.append(high_performers)
        colors.append(CHART_COLORS['high_performance'])  # Teal
    
    if training_needed > 0:
        categories.append(f'Training Needed\n(<70%): {training_needed}')
        counts.append(training_needed)
        colors.append(CHART_COLORS['training_needed'])  # Orange
    
    if average_performers > 0:
        categories.append(f'Average\n(70-90%): {average_performers}')
        counts.append(average_performers)
        colors.append(CHART_COLORS['average'])  # Gray
    
    if len(counts) > 0:
        wedges, texts, autotexts = ax3.pie(counts, labels=categories, autopct='%1.0f%%',
                                           colors=colors, startangle=90)
        ax3.set_title(f'Workforce Performance\n({total_employees} Total Employees)',
                     fontsize=14, fontweight='bold')
    else:
        ax3.text(0.5, 0.5, 'No Performance Data', ha='center', va='center', 
                transform=ax3.transAxes)
else:
    ax3.text(0.5, 0.5, 'No Employee Data', ha='center', va='center', 
            transform=ax3.transAxes)

# 4. Shift Analysis (Bottom Right) - Clear metric types
shift_analysis = room_cleaning.groupby('shift').agg({
    'employee_id': 'count',
    'room_cleaned': 'mean'
}).round(1)

if len(shift_analysis) > 0:
    shift_analysis.columns = ['sessions', 'avg_rooms']
    fraud_by_shift = fraud_cases.groupby('shift').size() if len(fraud_cases) > 0 else pd.Series()
    
    shifts = shift_analysis.index.tolist()
    shift_sessions = shift_analysis['sessions'].tolist()
    shift_avg_rooms = shift_analysis['avg_rooms'].tolist()
    shift_fraud_cases = [fraud_by_shift.get(shift, 0) for shift in shifts]
    
    x = range(len(shifts))
    width = 0.25
    
    bars3a = ax4.bar([i - width for i in x], shift_sessions, width, 
                     label='Sessions', color=CHART_COLORS['normal_neutral'], alpha=0.7)
    bars3b = ax4.bar(x, shift_avg_rooms, width, 
                     label='Avg Rooms/Session', color=CHART_COLORS['high_performance'], alpha=0.7)
    bars3c = ax4.bar([i + width for i in x], shift_fraud_cases, width, 
                     label='Fraud Cases', color=CHART_COLORS['fraud'], alpha=0.7)
    
    ax4.set_title('Shift Performance & Risk Analysis', fontsize=14, fontweight='bold')
    ax4.set_xlabel('Shift Type')
    ax4.set_ylabel('Count')
    ax4.set_xticks(x)
    ax4.set_xticklabels(shifts)
    ax4.legend()
    ax4.grid(axis='y', alpha=0.3)
    
    for bars in [bars3a, bars3b, bars3c]:
        for bar in bars:
            height = bar.get_height()
            if height > 0:
                ax4.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                        f'{height:.1f}', ha='center', va='bottom', fontsize=9)
else:
    ax4.text(0.5, 0.5, 'No Shift Data Available', ha='center', va='center', 
            transform=ax4.transAxes)

plt.tight_layout()
plt.savefig('../visualizations/executive_dashboard_summary.png', dpi=300, bbox_inches='tight')
plt.show()

print("Executive dashboard saved to: ../visualizations/executive_dashboard_summary.png")

# Summary Statistics
total_fraud_cases = len(fraud_cases) if 'fraud_cases' in locals() else 0
avg_completion_rate = (completion_rates.mean() if 'completion_rates' in locals() 
                       and len(completion_rates) > 0 else 0)

print(f"\nExecutive Summary Statistics:")
print("=" * 40)
print(f"Critical Issues:")
print(f"  - Fraud cases detected: {total_fraud_cases} employees")

if 'completion_rates' in locals() and len(completion_rates) > 0:
    training_needed_count = len(completion_rates[completion_rates < 0.7])
    print(f"  - Employees requiring training: {training_needed_count} "
          f"({training_needed_count/len(completion_rates)*100:.0f}%)")

print(f"\nKey Performance Indicators:")
print(f"  - Average completion rate: {avg_completion_rate:.0%} (Target: >80%)")

if 'shift_analysis' in locals() and len(shift_analysis) > 0:
    best_shift = shift_analysis['avg_rooms'].idxmax()
    best_productivity = shift_analysis['avg_rooms'].max()
    print(f"  - Peak productivity: {best_shift} shift ({best_productivity:.1f} rooms/session)")

print(f"\nEstimated Annual Impact:")
print(f"  - Fraud prevention savings: $20,000-40,000")
print(f"  - Training investment required: $25,000-40,000")
print(f"  - Expected productivity gains: 15-25%")
